# TITLE

In [1]:
import pickle
import random
import numpy as np
import pandas as pd
import networkx as nx
from pygsp import graphs, filters
from scipy.spatial import cKDTree

# PARAMETERS

In [2]:
#Parameters
N=10; #songs in the playlist
tau = 70 # heat diffusion coefficient
method = 2 # 0 -> KD-Tree | 1 -> 2D graphs coords | 2 -> Heat diffusion

# Paylists
meta_playlist = list()

# Data Loading

In [3]:
#Load data
dataSet = pd.read_pickle('data/dataSet.csv')
dfmax=np.load('data/dfmax.npy')
tracks = pd.read_csv('data/tracks.csv', index_col=0, header=[0, 1]);
tracks_raw = pd.read_csv('../data/fma_metadata/raw_tracks.csv', index_col=0);

In [4]:
# Loading custom songs features
features_c = pd.read_csv('data/custom_features.csv', index_col=0, header=[0, 1, 2], sep=',', encoding='utf-8')
tid_c = pd.read_csv('data/custom_tid.csv', index_col=0, header=[0], sep=',', encoding='utf-8')

features_c=features_c.divide(dfmax)
numb_cust=len(features_c)
features_c.head()

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
-1            0.000040 -0.000210 -0.000410 -0.000130  0.000480  0.000845   
-2            0.000793  0.001782  0.000439  0.000314  0.000445  0.000184   
-3           -0.000090 -0.000167 -0.000703 -0.000248 -0.000190 -0.000293   
-4            0.000045 -0.000121  0.000469 -0.000084 -0.000265 -0.000272   
-5            0.000001  0.000340  0.000633  0.000174 -0.000062  0.000008   

feature                                               ...      tonnetz  \
statistics                                            ...          std   
number            07        08        09        10    ...           04   
track_id                                              ...                
-1          0.000117 -0.000020  0.000198 -0.000022    ...     0.283183   
-2          0.000326  0.000688  0.000759  0.000755    ...     0.168852   
-3         -0.000254  0.000210  0.000169 -0.000311    ...     0.391788   
-4          0.000211  0.000181  0.000107 -0.000429    ...     0.350153   
-5         -0.000103 -0.000063  0.000072 -0.000255    ...     0.276001   

feature                              zcr                                \
statistics                      kurtosis       max      mean    median   
number            05        06        01        01        01        01   
track_id                                                                 
-1          0.201160  0.127417  0.000830  0.393258  0.083921  0.074114   
-2          0.135265  0.100488  0.000682  0.459697  0.097525  0.079564   
-3          0.238204  0.219284  0.006959  0.659013  0.043638  0.028883   
-4          0.251736  0.175649  0.008152  0.623840  0.049448  0.040327   
-5          0.222388  0.155338  0.000772  0.410357  0.099330  0.093188   

feature                                   
statistics       min      skew       std  
number            01        01        01  
track_id                                  
-1          0.000000  0.025869  0.110763  
-2          0.000000  0.026764  0.144817  
-3          0.018622  0.091895  0.144961  
-4          0.013035  0.075225  0.083216  
-5          0.000000  0.018609  0.084305  

[5 rows x 518 columns]

# Functions

In [5]:
# Dictionnary to link the node position and the track ID
idx_dict={i : dataSet.index[i] for i in range(len(dataSet.index))} # TODO loading/saving is useless actually    
inv_dict = {v: k for k, v in idx_dict.items()}

In [6]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, 'Open')+'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp'+'<a target="_blank" href="{}">{}</a>'.format(val+'/download', 'Download')

In [7]:
def display_result(playlist):
    result=pd.DataFrame({'Artist': tracks.artist.name[playlist], \
                         'Title':tracks.track.title[playlist], \
                         'Links':tracks_raw['track_url'][playlist]}) 

    # Output 
    result.reset_index(inplace=True)
    result.drop(columns='track_id',inplace=True)
    result = result.style.format({'Links':make_clickable})
    
    # Output
    display(result)

# KD-Tree

In [8]:
# Convert to np.array
points_ref = np.array([tuple(x) for x in dataSet.drop(columns='label',level=0).values])

# Construct the KD Tree
tree = cKDTree(points_ref)

## KD-Tree based

In [9]:
#list of the tracks nearest to each custom song
nearest_tracks=dict()

# sub-meta playlist
over_playlist = list()

# For each Playlist we find the nearest point in the KD-Tree with the given songs
for name in tid_c['Playlist name'].unique():
    tid = np.array(tid_c[tid_c['Playlist name'] == name].index.values)
    
    # TODO Tim
    features_c_t = np.array([features_c[features_c.index == idx].values[0] for idx in tid])
    
    # Find the 10th nearest points with Euclidean norm in the tree
    dist, idx = tree.query(features_c_t, k=N, p=2)
    
    # We get the correspond Track ID from the position IDX
    playlist = list([idx_dict[node] for node in idx.ravel()])
    
    # For each song in the playlist we get the best match and store it for later use
    top_tracks = [idx_dict[x] for x in idx[:,0]]
    nearest_tracks.update({name:top_tracks})
    
    # Select in the closest neighbours N random tracks
    playlist = random.sample(playlist,N)
    
    # Add to sub-meta playlist
    over_playlist.append(playlist)

#
meta_playlist.append(over_playlist)

## 2-D coords based

In [10]:
# Create the KD-Tree from the graphs coordinates
coords2D = np.load("data/coords2D.npy").item()

graphCoords=list()
for i in coords2D:
    graphCoords.append(tuple(coords2D[i]))

graphCoords=np.array(graphCoords)
graphTree = cKDTree(graphCoords)

In [11]:
over_playlist = list()

# For each playlist, access nearest tracks
for name in tid_c['Playlist name'].unique():
    
    playlist=list()
    # Retrieve the closest track ID for the given playlist
    tid = nearest_tracks[name]
    
    # Find closest tracks in the graph for a given custom song
    dist, idx = graphTree.query([tuple(coords2D[x]) for x in tid], k=N, p=2)
    
    # Create the playlist
    playlist = list([idx_dict[node] for node in idx.ravel()])

    # Select in the closest neighbours N random tracks
    playlist=random.sample(playlist,N)

    # Add to sub-meta playlist
    over_playlist.append(playlist)
    
#
meta_playlist.append(over_playlist)

## Heat Diffusion

In [12]:
# Construct Graph
conn_graph = np.load("data/connGraph.npy")
G = graphs.Graph(conn_graph)
G.compute_fourier_basis()
G.set_coordinates()

In [13]:
over_playlist = list()

# For each playlist, access nearest tracks
for name in tid_c['Playlist name'].unique():

    # Map the nearest track IDs to the nodes position
    musics_pos = [inv_dict[tid] for tid in nearest_tracks[name]]

    list_heat_neighbour = np.array([]).astype(int)
    s_out_moy = np.zeros(G.N)

    for i in musics_pos:
        s = np.zeros(G.N)
        s[i]=1

        g = filters.Heat(G, tau, normalize=False)
        s_out = g.filter(s, method='chebyshev')
        s_out = s_out/max(s_out)

        s_out_moy = s_out_moy + s_out

    ind_max_heat = np.argsort(s_out_moy)
    list_heat_neighbour= np.append(list_heat_neighbour,ind_max_heat[-N:])
    
    # Create the playlist
    playlist = [idx_dict[idx] for idx in list_heat_neighbour]
    
    # Select in the closest neighbours N random tracks
    playlist = random.sample(playlist,N)

    # Add to sub-meta playlist
    over_playlist.append(playlist)
    
# 
meta_playlist.append(over_playlist)

# Output

In [14]:
for i,playlist in enumerate(tid_c['Playlist name'].unique()):
    print('Playlist:', playlist)
    display_result(meta_playlist[method][i])
    
# Download la playlist entière! TODO

Playlist: Tim


,Artist,Title,Links
0,Laura Stevenson and the Cans,Landslide Song/The Dig,Open Download
1,Leif Vollebek,Photographer Friend,Open Download
2,Big Blood,The WInds House,Open Download
3,AWOL,Food,Open Download
4,Lonely Faction,Softens The Blow,Open Download
5,Keaton Henson,You Don't Know How Lucky You Are,Open Download
6,Laura Stevenson and the Cans,Nervous Rex,Open Download
7,Tony Gage,80s Action Movie,Open Download
8,Special Moments,They Dropped a Bomb (On Me),Open Download
9,Julia K. Mars,The New Chance,Open Download


Playlist: Anael


,Artist,Title,Links
0,Albin Andersson,Even If It's True,Open Download
1,Milk Music,Thrashing In The Unknown,Open Download
2,The Planes,Sadie (4-Track Demo),Open Download
3,Laura Stevenson and the Cans,Baby Bones,Open Download
4,The Woolen Men,Chinese Rug,Open Download
5,Brett Saxon,One Thing,Open Download
6,The Impossebulls,Time Is Running Away (Destroy Everything),Open Download
7,Scott Holmes,Reflections Across The Sky,Open Download
8,Neutral Fixation,"""I'm Eating Plastic""",Open Download
9,The Dread,Permission,Open Download


Playlist: Lucas


,Artist,Title,Links
0,LOWdown,So On and Yo!,Open Download
1,Josh Mease,Overboard,Open Download
2,Jahzzar,Wastecnology,Open Download
3,Mutilation Rites,Suffer The Children,Open Download
4,Strapping Fieldhands,In the Pineys,Open Download
5,The Twin Atlas,Happy At The Wheel,Open Download
6,Bloodgod,Heist,Open Download
7,Upsilon Acrux,So Thereby...and furthermore...thus henceforth...Mono,Open Download
8,Tommy Jay,Memories,Open Download
9,Josh Woodward,Omaha (No Vocals),Open Download


Playlist: Joacqim


,Artist,Title,Links
0,Kellee Maize,Big Plans (Remix),Open Download
1,Flamin' Groovies,Shake Some Action,Open Download
2,Apache Tomcat,Nothing On But The Radio,Open Download
3,Ed Askew,When I Arive,Open Download
4,Westy Reflector,i must have love,Open Download
5,C-Doc,Life Slow Down (Vocal) (Featuring Jamod Allah),Open Download
6,Alexander Turnquist,Finding The Butterfly,Open Download
7,Fallen to Flux,Relapse,Open Download
8,Sic Alps,Message From the Law,Open Download
9,Blank & Kytt,Thursday & Snow (Reprise),Open Download
